In [1]:
import os
import argparse
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.preprocessing import OrdinalEncoder
from sklearn.calibration import CalibratedClassifierCV
from scipy.sparse import hstack

import glob
import numpy as np

import matplotlib.pyplot as plt
from sklearn.calibration import CalibrationDisplay
from sklearn.metrics import roc_auc_score
import seaborn as sns

from collections import Counter

In [2]:
path = f"C:/Users/hwallace/glaccount/results"

In [3]:
all_files = glob.glob(os.path.join(path, '*.xlsx'))

In [4]:
all_files

['C:/Users/hwallace/glaccount/results\\1259 GL Account Final.xlsx',
 'C:/Users/hwallace/glaccount/results\\2022.06.14 gl_accounts existing clients.xlsx',
 'C:/Users/hwallace/glaccount/results\\GL Accounts new 7.7.22.xlsx',
 'C:/Users/hwallace/glaccount/results\\GL New Accounts 10.13.22.xlsx',
 'C:/Users/hwallace/glaccount/results\\GL New Accounts 11.4.22.xlsx',
 'C:/Users/hwallace/glaccount/results\\GL New Accounts 8.11.22.xlsx',
 'C:/Users/hwallace/glaccount/results\\GL New Accounts 9.14.22.xlsx',
 'C:/Users/hwallace/glaccount/results\\GL New Clients 10.24.22.xlsx',
 'C:/Users/hwallace/glaccount/results\\GL New Clients 10.4.22.xlsx',
 'C:/Users/hwallace/glaccount/results\\GL New Clients 5.12.22.xlsx',
 'C:/Users/hwallace/glaccount/results\\GL New Clients 6.1.22.xlsx',
 'C:/Users/hwallace/glaccount/results\\gl_accounts new 7.8.22.xlsx',
 'C:/Users/hwallace/glaccount/results\\gl_accounts_202206 ji edits.xlsx',
 'C:/Users/hwallace/glaccount/results\\gl_accounts_2734_2957 _8.1.22.xlsx',
 

In [5]:
len(all_files)

23

In [17]:
dd = {}
d = {}
for i, file in enumerate(all_files):
    dd[i] = pd.read_excel(file, engine="openpyxl", sheet_name=None)
    d[i] =  pd.concat(dd[i].values(), ignore_index=True)
    d[i] = d[i].rename({"Jen's Review": 'Review'}, axis=1)
    d[i].loc[d[i]['LineItem'] == d[i]['Review'], 'Review'] = np.NaN

In [ ]:
# d = {}
# for i, item in enumerate(dd.values()):
#     d[i] = pd.concat(item.values(), ignore_index=True)

In [ ]:
# for i, item in enumerate(d.values()):
#     item.rename({"Jen's Review": 'Review'}, axis=1, inplace=True)

In [ ]:
# d[0].loc[d[0]['LineItem'] == d[0]["Jen's Review"], "Jen's Review"] = pd.NA

In [19]:
l = []
for i in range(len(d)):
    l = l + list(d[i].columns)

In [20]:
Counter(l)

Counter({'OrgPin': 23,
         'AccountID': 23,
         'AccountCode': 23,
         'Description': 23,
         'GLRollup': 23,
         'GLDollars': 23,
         'LineItem': 23,
         'Review': 23,
         'SPAccountRollupCategory': 23,
         'SPAccountRollupName': 23,
         'OBDollarsFinancialReporting': 23,
         'DSSAccountRollup1Name': 23,
         'Category': 23,
         'Section': 23,
         'Statement': 23,
         'IsDollarF': 13,
         'IsStat': 13,
         'OrgAccntID': 1,
         'OBDollarsFinancialReportingCategory': 1})

In [25]:
## 10.24 has one org with nothing
## 10.4 has one org with nothing 1259
## 5.05 has nothing in any of the orgs
## 7.8 and ji edits both have the same number of rows (?)
## ji edits seems to be correct

In [21]:
col = "Review"
for i in range(len(d)):
    print(f'file # {i}')
    print(f'file {all_files[i]}')
    print(f'length: {len(d[i])}')
    print(f'review nulls: {d[i][col].isna().sum()/len(d[i])}')
    print('')

file # 0
file C:/Users/hwallace/glaccount/results\1259 GL Account Final.xlsx
length: 2278
review nulls: 0.43239683933274803

file # 1
file C:/Users/hwallace/glaccount/results\2022.06.14 gl_accounts existing clients.xlsx
length: 955
review nulls: 0.5884816753926702

file # 2
file C:/Users/hwallace/glaccount/results\GL Accounts new 7.7.22.xlsx
length: 19005
review nulls: 0.7695343330702447

file # 3
file C:/Users/hwallace/glaccount/results\GL New Accounts 10.13.22.xlsx
length: 688
review nulls: 0.5973837209302325

file # 4
file C:/Users/hwallace/glaccount/results\GL New Accounts 11.4.22.xlsx
length: 394
review nulls: 0.631979695431472

file # 5
file C:/Users/hwallace/glaccount/results\GL New Accounts 8.11.22.xlsx
length: 1711
review nulls: 0.33489187609585036

file # 6
file C:/Users/hwallace/glaccount/results\GL New Accounts 9.14.22.xlsx
length: 1517
review nulls: 0.26697429136453527

file # 7
file C:/Users/hwallace/glaccount/results\GL New Clients 10.24.22.xlsx
length: 292
review nulls:

In [23]:
d[11].sort_values(['OrgPin', 'AccountID']).head()

,OrgPin,AccountID,AccountCode,Description,GLRollup,GLDollars,LineItem,Review,SPAccountRollupCategory,SPAccountRollupName,OBDollarsFinancialReporting,DSSAccountRollup1Name,Category,Section,Statement
6707,1130,532,306,Directed Payment,Revenues,-2.727585e+08,Other Operating Revenue,NaN,Payor Related Accounts Category,Contractuals,IS - Allowances-Adjustments: - Contractuals an...,Excluded,Revenue - Other - Operating,Revenue,Income Statement
6709,1137,8200,4300-0190-00,Deductions from Revenue - Inpatient-Deductions...,Deductions From Revenue-IP,-7.378535e+04,Contractual Allowances,NaN,Payor Related Accounts Category,Contractuals,IS - Deductions from Rev - Deductions From Rev...,Excluded,Revenue Deduction,Revenue,Income Statement
2236,1137,8202,6400-0083-00,Drug Costs-TV Cable and Internet Serv,Other Expenses,-5.188200e+02,Drugs,Other Expenses,Expenses - Operating,Other Expenses,IS - Operating Expenses - Other Expenses,Other Expenses,Expenses - Operating,Expense,Income Statement
2237,1137,8203,7050-0057-00,Equipment Expense-Non Capital Medical Equipment,Other Expenses,6.296800e+03,Equipment,Other Expenses,Expenses - Operating,Other Expenses,IS - Operating Expenses - Other Expenses,Other Expenses,Expenses - Operating,Expense,Income Statement
2238,1137,8204,7100-0183-00,Travel Training and Meeting-Dietary - Food and...,Other Expenses,-5.005000e+01,Other Expenses - Food,NaN,Expenses - Operating,Other Expenses,IS - Operating Expenses - Other Expenses,Other Expenses,Expenses - Operating,Expense,Income Statement


In [24]:
d[12].sort_values(['OrgPin', 'AccountID']).head()

,OrgAccntID,OrgPin,AccountID,AccountCode,Description,GLRollup,GLDollars,LineItem,Review,SPAccountRollupCategory,SPAccountRollupName,OBDollarsFinancialReporting,DSSAccountRollup1Name,Category,Section,Statement
0,1130532,1130,532,306,Directed Payment,Revenues,-2.727585e+08,Other Operating Revenue,Contractual Allowances,Payor Related Accounts Category,Contractuals,IS - Allowances-Adjustments: - Contractuals an...,Excluded,Revenue - Other - Operating,Revenue,Income Statement
2094,11378200,1137,8200,4300-0190-00,Deductions from Revenue - Inpatient-Deductions...,Deductions From Revenue-IP,-7.378535e+04,Contractual Allowances,NaN,Payor Related Accounts Category,Contractuals,IS - Deductions from Rev - Deductions From Rev...,Excluded,Revenue Deduction,Revenue,Income Statement
2093,11378202,1137,8202,6400-0083-00,Drug Costs-TV Cable and Internet Serv,Other Expenses,-5.188200e+02,Drugs,Utilities,Expenses - Operating,Other Expenses,IS - Operating Expenses - Other Expenses,Other Expenses,Expenses - Operating,Expense,Income Statement
1,11378203,1137,8203,7050-0057-00,Equipment Expense-Non Capital Medical Equipment,Other Expenses,6.296800e+03,Equipment,NaN,Expenses - Operating,Other Expenses,IS - Operating Expenses - Other Expenses,Other Expenses,Expenses - Operating,Expense,Income Statement
2,11378204,1137,8204,7100-0183-00,Travel Training and Meeting-Dietary - Food and...,Other Expenses,-5.005000e+01,Other Expenses - Food,NaN,Expenses - Operating,Other Expenses,IS - Operating Expenses - Other Expenses,Other Expenses,Expenses - Operating,Expense,Income Statement


In [36]:
dcomp = d[11].merge(d[12], on=['OrgPin', 'AccountID']).fillna('None')

In [39]:
dcomp.loc[(dcomp.Review_x != dcomp.Review_y) & (dcomp.OrgPin == 1152), ['OrgPin', 'AccountID', 'LineItem_x', 'LineItem_y', 'Review_x', 'Review_y']]

,OrgPin,AccountID,LineItem_x,LineItem_y,Review_x,Review_y
0,1152,599877,Long Term Liabilities,Long Term Liabilities,Contractual Allowances,None
6,1152,599097,Long Term Liabilities,Long Term Liabilities,Benefits - Workers Comp,None
7,1152,599073,Long Term Liabilities,Long Term Liabilities,Benefits - Workers Comp,None
8,1152,600050,Long Term Liabilities,Long Term Liabilities,Benefits - Other,None
9,1152,598788,Long Term Liabilities,Long Term Liabilities,Benefits - Other,None
...,...,...,...,...,...,...
2083,1152,600321,Utilities,Utilities,Other Expenses,None
2084,1152,857561,Utilities,Utilities,Other Expenses,None
2085,1152,600380,Utilities,Utilities,Other Expenses,None
2086,1152,600176,Utilities,Utilities,Other Expenses,None


In [40]:
dcomp.loc[(dcomp.Review_x != dcomp.Review_y) & (dcomp.OrgPin == 1152)  & (dcomp.Review_y != 'None'), ['OrgPin', 'AccountID', 'LineItem_x', 'LineItem_y', 'Review_x', 'Review_y']]

,OrgPin,AccountID,LineItem_x,LineItem_y,Review_x,Review_y


In [41]:
dcomp.loc[(dcomp.Review_x != dcomp.Review_y) & (dcomp.Review_y != 'None'), ['OrgPin', 'AccountID', 'LineItem_x', 'LineItem_y', 'Review_x', 'Review_y']]

,OrgPin,AccountID,LineItem_x,LineItem_y,Review_x,Review_y
2095,3220,3027,Supplies,Supplies,Accounts Receivable - Allowance for Doubtful A...,Charity Care
2100,1430,3083,Long Term Liabilities,Long Term Liabilities,Accounts Receivable - Contractual Allowance,Contractual Allowances
2125,3121,10197,Supplies,Supplies,Accounts Receivable - Other,Accounts Payable
2127,3121,10283,Intercompany Receivables,Intercompany Receivables,Accounts Receivable - Other,Accounts Receivable - Due from Related Parties
2128,3121,9694,Other Expenses,Other Expenses,Accounts Receivable - Other,Accounts Receivable - Due from Related Parties
...,...,...,...,...,...,...
6729,2724,2058,Not Specified,Not Specified,None,Salaries
6730,1829,2994,Long Term Liabilities,Long Term Liabilities,None,Salaries - Incentive
6731,1166,37134,Long Term Liabilities,Long Term Liabilities,None,Contract Labor
6732,1430,3101,Long Term Liabilities,Long Term Liabilities,None,Salaries - Incentive


In [ ]:
# use database P_FB8AD0C32C6A4EBBA7ED45F877040C97;

# select accountid, sphlineitem 
# from FW.DIMACCOUNT 
# where accountid in (599877, 599097, 599073, 600050, 598788) ;

# select accountid, sphlineitem 
# from FW.DIMACCOUNT 
# where accountid in (600321, 857561, 600380, 600176, 857563) ;

In [42]:
dcomp.loc[(dcomp.Review_x != dcomp.Review_y) & (dcomp.OrgPin == 1152)].groupby('Review_x').size()

Review_x
Accounts Payable                                           2
Accounts Payable - Accrued Payroll                         1
Accounts Receivable (Gross)                                3
Accounts Receivable - Allowance for Doubtful Accounts      1
Accounts Receivable - Contractual Allowance                1
                                                        ... 
Supplies - Office                                         12
Supplies - Paper                                           5
Temporarily Restricted Net Assets                        180
Unable to Categorize                                       2
Utilities                                                  6
Length: 61, dtype: int64

In [43]:
dcomp.loc[(dcomp.Review_x != dcomp.Review_y) & (dcomp.OrgPin == 1152)].groupby('Review_y').size()

Review_y
None    1745
dtype: int64

In [20]:
df = pd.concat((pd.read_excel(f, engine="openpyxl") for f in all_files), ignore_index=True)

In [22]:
df.groupby('OrgAccntID').size()

OrgAccntID
3.186680e+05    1
1.130532e+06    1
1.770786e+06    1
1.770798e+06    1
1.770799e+06    1
               ..
1.195103e+09    1
1.421737e+09    1
1.421737e+09    1
1.421737e+09    1
1.421737e+09    1
Length: 6735, dtype: int64

In [24]:
df.groupby('OBDollarsFinancialReportingCategory').size()

OBDollarsFinancialReportingCategory
Corporate Allocations    22
Investment in WMG        10
dtype: int64

In [27]:
df.loc[df['OBDollarsFinancialReportingCategory'] == 'Investment in WMG']

,OrgPin,AccountID,AccountCode,Description,GLRollup,GLDollars,LineItem,Jen's Review,SPAccountRollupCategory,SPAccountRollupName,OBDollarsFinancialReporting,DSSAccountRollup1Name,Category,Section,Statement,IsDollarF,IsStat,OrgAccntID,OBDollarsFinancialReportingCategory
110746,9868,2295,470000301,WMG AHP NET MARGIN REDIRECT,Investment in WMG,0.00,Other Operating Revenue,Other Expenses,Expenses - Corporate Allocations,WMG Investment,IS - Investment in WMG - Investment in WMG,Not Specified,Revenue - Other - Operating,Revenue,Income Statement,NaN,NaN,NaN,Investment in WMG
110747,9868,2294,470000302,CVM AHP NET MARGIN REDIRECT,Investment in WMG,0.00,Other Operating Revenue,Other Expenses,Expenses - Corporate Allocations,WMG Investment,IS - Investment in WMG - Investment in WMG,Not Specified,Revenue - Other - Operating,Revenue,Income Statement,NaN,NaN,NaN,Investment in WMG
110748,9868,2297,470010000,CVM ANCILLARY REV REDIRECT,Investment in WMG,0.00,Other Operating Revenue,Other Expenses,Expenses - Corporate Allocations,WMG Investment,IS - Investment in WMG - Investment in WMG,Not Specified,Revenue - Other - Operating,Revenue,Income Statement,NaN,NaN,NaN,Investment in WMG
110749,9868,2298,470020000,RR WMG BUD CORP SRVS,Investment in WMG,-0.01,Other Operating Revenue,Other Expenses,Expenses - Corporate Allocations,WMG Investment,IS - Investment in WMG - Investment in WMG,Not Specified,Revenue - Other - Operating,Revenue,Income Statement,NaN,NaN,NaN,Investment in WMG
110750,9868,3191,470030000,RR WMG BUD NET MAR,Investment in WMG,0.01,Other Operating Revenue,Other Expenses,Expenses - Corporate Allocations,WMG Investment,IS - Investment in WMG - Investment in WMG,Not Specified,Revenue - Other - Operating,Revenue,Income Statement,NaN,NaN,NaN,Investment in WMG
110751,9868,3713,470040000,WMG ECHP NET MARGIN REDIRECT,Investment in WMG,0.00,Other Operating Revenue,Other Expenses,Expenses - Corporate Allocations,WMG Investment,IS - Investment in WMG - Investment in WMG,Not Specified,Revenue - Other - Operating,Revenue,Income Statement,NaN,NaN,NaN,Investment in WMG
110752,9868,4853,470050000,WMG PIC NET MARGIN REDIRECT,Investment in WMG,0.00,Other Operating Revenue,Other Expenses,Expenses - Corporate Allocations,WMG Investment,IS - Investment in WMG - Investment in WMG,Not Specified,Revenue - Other - Operating,Revenue,Income Statement,NaN,NaN,NaN,Investment in WMG
110753,9868,5550,470060000,WMG VHP NET MARGIN REDIRECT,Investment in WMG,0.00,Other Operating Revenue,Other Expenses,Expenses - Corporate Allocations,WMG Investment,IS - Investment in WMG - Investment in WMG,Not Specified,Revenue - Other - Operating,Revenue,Income Statement,NaN,NaN,NaN,Investment in WMG
110754,9868,7054,470070000,WMG CHP NET MARGIN REDIRECT,Investment in WMG,0.00,Other Operating Revenue,Other Expenses,Expenses - Corporate Allocations,WMG Investment,IS - Investment in WMG - Investment in WMG,Not Specified,Revenue - Other - Operating,Revenue,Income Statement,NaN,NaN,NaN,Investment in WMG
110755,9868,7053,470080000,WMG AVHP NET MARGIN REDIRECT,Investment in WMG,0.00,Other Operating Revenue,Other Expenses,Expenses - Corporate Allocations,WMG Investment,IS - Investment in WMG - Investment in WMG,Not Specified,Revenue - Other - Operating,Revenue,Income Statement,NaN,NaN,NaN,Investment in WMG


In [28]:
df.groupby("Jen's Review").size()

Jen's Review
Accounts Payable                                          87
Accounts Payable - Accrued Interest Payable                3
Accounts Payable - Accrued Payroll                         2
Accounts Receivable (Gross)                              213
Accounts Receivable - Allowance for Doubtful Accounts     88
                                                        ... 
Temporarily Restricted Earnings                            3
Temporarily Restricted Net Assets                         10
Unable to Categorize                                     365
Unrestricted Net Assets                                  108
Utilities                                                 79
Length: 173, dtype: int64